#Download dataset (images and labels) from Open Image Dataset V7 , YOLOv8 format

In [ ]:
!git clone https://github.com/NanoCode012/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format.git

In [ ]:
!pip3 install -r /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/requirements.txt --quiet

In [3]:
!mkdir OID
!mkdir OID/Dataset

#python main.py downloader --classes <class names> --type_csv <train/test/validation/all> --limit <number of image from each class>

## By adding --yoloLabelStyle to the command above, the output labels will be the yolo format, but if you don' add this switch, You have to change the format to the proper one.

In [4]:
classes = 'Apple Banana'
samples = 300

!python /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/main.py downloader --classes {classes} --type_csv train --limit {samples} --yoloLabelStyle
#!python /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/main.py downloader --classes {classes} --type_csv train --limit {samples} --yoloLabelStyle --multiclasses 1

    [INFO] |  saving dataset configurations at ./OID/Dataset/config.json

		   ___   _____  ______            _    _       _    _  
		 .'   `.|_   _||_   _ `.         | |  | |     | |  | |    
		/  .-.  \ | |    | | `. \ _   __ | |__| |_    |  | | |   
		| |   | | | |    | |  | |[ \ [  ]|____   _|     | | | 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_       | |
		 `.___.'|_____||______.'   \__/     |_____|     |_|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Apple.
   [ERROR] | Missing the class-descriptions-boxable.

#Let’s visualize our Data

##

The ToolKit is useful also for visualize the downloaded images with the respective labels.

   ##python3 main.py visualizer
In this way the default Dataset folder will be pointed to search the images and labels automatically. To point another folder it's possible to use --Dataset optional argument.

   ##python3 main.py visualizer --Dataset desired_folder
Then the system will ask you which folder to visualize (train, validation or test) and the desired class. Hence with d (next), a (previous) and q (exit) you will be able to explore all the images. Follow the menu for all the other options.

In [8]:
#!python3 main.py visualizer   #In this way the default Dataset folder will be pointed to search the images and labels automatically.
!python3 /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/main.py visualizer --Dataset /content/OID/Dataset  #To point another folder it's possible to use --Dataset optional argument.

    [INFO] |  try to laod dataset configurations at /content/OID/Dataset/config.json

		   ___   _____  ______            _    _       _    _  
		 .'   `.|_   _||_   _ `.         | |  | |     | |  | |    
		/  .-.  \ | |    | | `. \ _   __ | |__| |_    |  | | |   
		| |   | | | |    | |  | |[ \ [  ]|____   _|     | | | 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_       | |
		 `.___.'|_____||______.'   \__/     |_____|     |_|
	
 
            _    _ _                  _ _                  
           | |  | (_)                | (_)                 
           | |  | |_  ___ _   _  ____| |_ _____ ____  ____ 
            \ \/ /| |/___) | | |/ _  | | (___  ) _  )/ ___)
             \  / | |___ | |_| ( ( | | | |/ __( (/ /| |    
              \/  |_(___/ \____|\_||_|_|_(_____)____)_|    
                                                                                                                                                                                                    
        

#Let’s download the data to our PC

In [ ]:
!zip -r /content/data.zip  /content/OID/Dataset/train/

# Converting the annotations to the YOLO format which is:

## **name_of_the_class    left    top     right     bottom**

 (https://github.com/EscVM/OIDv4_ToolKit)

but yolo requires this:

## **Yolo format ===>  name_of_class  x  y  width  height**
where

**name_of_class**: Class name for the bounding boxes of the object

**x**: The x coordinate of the centre of the rectangle

**y**: The y coordinate of the centre of the rectangle

**width**: width of the rectangle

**height**: height of the rectangle.

###
Changing the open image annotation to YOLO annotation:
The open image dataset annotation has to be changed into the YOLO format before it can be used in the YOLO model training.

First, let's understand the math involved in annotation transformation.

We have the coordinates of the top left corner of the called represented as (left, top) and right bottom corner as (right, bottom).

we can find the width of the image by subtracting the left from right. and similarly height by subtracting the top from bottom.

width = right-left

height = bottom - top

Now the centre of the rectangle will be the midpoint of width and the height. hence x and y can be calculated as:

x = left + (width / 2)

y = top + (height / 2)

now we have all the coordinates required for the YOLO annotations i.e. x,y, width, height. we can use these for YOLO object detection.





In [ ]:
import os
import glob
import cv2

# Specify the directory containing the text files
root_directory = '/content/OID/Dataset/train/'  # Update this path
classes = ['Apple', 'Banana']
for aclass in classes:
    label_directory = os.path.join(root_directory, f'{aclass}/labels/')
    image_directory = os.path.join(root_directory, f'{aclass}/images/')
    image_size = []
    for imagepath in glob.glob(os.path.join(image_directory, '*.jpg')):
        # Open the image and read its size
        #with open(imagepath, 'r') as image:
          image = cv2.imread(imagepath)
          image_size.append(image.shape)

    # Iterate over all text files in the specified directory
    for num,filepath in enumerate(glob.glob(os.path.join(label_directory, '*.txt'))):
        image_h = image_size[num][0]
        image_w = image_size[num][1]
        # Open the file and read its content
        with open(filepath, 'r') as file:
            lines = file.readlines()

        # Modify each line as needed
        modified_lines = []
        for line in lines:
            words = line.split()
            if words:
                if words[0].lower() == 'apple':
                    words[0] = '1'
                elif words[0].lower() == 'banana':
                    words[0] = '0'


                # Convert coordinates from string to float
                left = float(words[1])
                top = float(words[2])
                right = float(words[3])
                bottom = float(words[4])

                # Calculate width and height
                width = right - left
                height = bottom - top

                # Calculate the center coordinates
                x = left + (width / 2)
                y = top + (height / 2)

                # Normalizing the coordinates
                x_norm = x / image_w
                y_norm = y / image_h
                width_norm = width / image_w
                height_norm = height/image_h

                # Update the words list with the new values
                words[1] = str(x_norm)
                words[2] = str(y_norm)
                words[3] = str(width_norm)
                words[4] = str(height_norm)

            modified_lines.append(' '.join(words))

        # Write the modified content back to the file
        with open(filepath, 'w') as file:
            file.write('\n'.join(modified_lines))

        print(f"Processed file: {filepath}")


# Refrences
https://sharmaji27.medium.com/easiest-way-to-download-data-from-the-open-image-dataset-553dccfb92d8

https://g-vj.medium.com/custom-image-dataset-for-yolo-object-detection-using-open-image-dataset-41c128b9231b

https://github.com/ultralytics/ultralytics

https://github.com/monocongo/openimages

https://www.basic.ai/blog-post/data-annotation-for-yolo-model-training-techniques-and-best-practices-1

https://docs.ultralytics.com/tasks/detect/#predict

https://github.com/EscVM/OIDv4_ToolKit
